# TP rectification homographique

**Guillaume Bourmaud - guillaume.bourmaud@enseirb-matmeca.fr**

L'objectif de ce TP est de mettre en oeuvre la notion d'homographie étudiée en cours.

Pour cela, nous allons considérer l'image suivante ([lien de téléchargement](https://gbourmaud.github.io/files/vision/TP/TP_rectification_homographique/image1.jpg)) :  
  
![alt text](image1.jpg)
Cette image correspond à la projection d'une scène planaire (dans notre cas une feuille A4 posée sur le sol) dans le plan image d'une caméra idéale du point de vue du modèle sténopé. Dans cette image, le texte présent sur la feuille A4 est difficilement lisible en raison de la rotation et de la translation de la caméra vis-à-vis de la feuille lors de la prise de vue.  
  
Nous avons vu que la transformation entre deux images $\mathbf{I}_A$ et $\mathbf{I}_B$ issues d'une scène planaire est une homographie $\mathbf{H}_{AB}$ :  

$$\underline{\mathbf{p}}_{A,i}=\pi(\mathbf{H}_{AB} \underline{\mathbf{p}}_{B,i}),$$  
  
où $\underline{\mathbf{p}}_{A,i}$ est un point (en coordonnées homogènes) dans l'image A, $\underline{\mathbf{p}}_{B,i}$ est son correspondant dans l'image B et $\pi(\cdot)$ est la fonction de projection.

Ce TP a pour but de mettre en oeuvre une méthode de rectification homographique permettant d'estimer cette matrice d'homographie $\mathbf{H}_{AB}$. Une fois estimée, cette matrice définit une transformation qui pourra être appliquée à l'image ci-dessus pour obtenir une **image rectifiée** où le texte présent sur la feuille A4 est lisible. 
  
Après rectification homographique, l'image rectifiée doit représenter la feuille A4 de telle sorte que cette dernière soit parfaitement lisible :  
  
![image rectifiée](image1_rect.jpg)

### Correspondances

Afin d'estimer l'homographie $\mathbf{H}_{AB}$, nous avons besoin de 4 correspondances $(\mathbf{p}_{A,i}, \mathbf{p}_{B,i})_{i=1...4}$ définies graphiquement ci-après :

![schéma](schema.png)

L'image B représente une feuille A4 de taille 21cm par 29,7cm. Quelles sont les coordonnées (en cm) des 4 coins $(\mathbf{p}_{B,i})_{i=1...4}$ ? Dans votre code, choisissez une résolution, par exemple 1 cm = 10 pixels, et créez une variable contenant les coordonnées en pixels :



In [ ]:
p_B = np.array([[..., ..., ..., ...],[..., ..., ..., ...]])

Concernant l'image A, nous pourrions faire une détection automatique des 4 coins $(\mathbf{p}_{A,i})_{i=1...4}$ mais ce n'est pas l'objet de ce TP. À la place, vous pouvez simplement récupérer manuellement les coordonnées des 4 coins $(\mathbf{p}_{A,i})_{i=1...4}$, puis créer dans votre code une variable contenant ces coordonnées :



In [ ]:
p_A = np.array([[..., ..., ..., ...],[..., ..., ..., ...]])

### Vérification visuelle des 4 correspondances

La plupart des bugs proviennent de mauvaises correspondances. Ainsi dans votre code, reproduisez le schéma ci-dessus  (l'image B sera blanche dans votre cas car c'est l'image que vous souhaitez obtenir) pour vous assurer que vos correspondances sont bonnes.


### Implémentation de la méthode d'estimation ("DLT")

1. Coder la méthode d'estimation d'homographie vue en cours.  
    A.  Construire la matrice $\mathbf{M}$ de taille $8\times8$  et le vecteur $\mathbf{p}_A$ de taille $8\times1$  
    B.  Résoudre numériquement (fonction `np.linalg.solve`) le système linéaire $\mathbf{M}^T \mathbf{M}\mathbf{h}=\mathbf{M}^T \mathbf{p}_A$ (voir [Annexe Moindres carrés linéaires](https://gbourmaud.github.io/files/vision/annexes/annexe_LS.pdf))   
    C.  Construire la matrice d'homographie $\mathbf{H}_{AB}$ de taille $3\times3$ à partir du vecteur $\mathbf{h}$  
    D.  Vérifier que $\underline{\mathbf{p}}_{A,i} = \pi(\mathbf{H}_{AB}\underline{\mathbf{p}}_{B,i})$ $\forall i\in [1...4]$ en traçant une croix colorée pour chaque $\pi(\mathbf{H}_{AB}\underline{\mathbf{p}}_{B,i})$ dans la figure précédente.
5. Appliquer l'homographie estimée $\mathbf{H}_{AB}$ à l'image à rectifier (application d'une transformation à une image).
6. Sauvegarder l'image rectifiée. Le document obtenu devrait être parfaitement lisible.

**SOLUTION : vous trouverez à la fin de ce fichier le script complet auquel vous pourrez vous référer si vous êtes bloqués.**

### Fonction `findHomography` d'OpenCV


La fonction `findHomography` d'OpenCV (`import cv2`) permet d'estimer une homographie à partir de correspondances. Tester cette fonction, et vérifier que le résultat obtenu est identique au vôtre. 

## CORRECTION SCRIPT COMPLET

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from getInterpolationGrids import getInterpolationGrids
from interp2_bilinear import interp2_bilinear
import cv2

im = np.array(Image.open('./image1.jpg')).astype(float)/255.

f1 = plt.figure()
ax1 = f1.add_subplot(121)
ax1.imshow(im)

#coordonnées des coins de la feuille dans l'image
#ordre : haut gauche, haut droit, bas droit, bas gauche
p_A = np.array([[305.,831.,3671.,4293.],[2238.,244.,211.,2219.]])

markersize = 30
ax1.scatter(p_A[0,0],p_A[1,0],ec='r', s=markersize, fc='none')
ax1.scatter(p_A[0,1],p_A[1,1],ec='g', s=markersize, fc='none')
ax1.scatter(p_A[0,2],p_A[1,2],ec='b', s=markersize, fc='none')
ax1.scatter(p_A[0,3],p_A[1,3],ec='k', s=markersize, fc='none')

ax1.set(
       title='Image prise')
#coordonnées des coins de la feuille idéale format A4
#ordre : haut gauche, haut droit, bas droit, bas gauche
scale = 100
h_rec = int(29.7*scale)
w_rec = int(21.*scale)
p_B = np.array([[0., w_rec-1, w_rec-1, 0.],[0., 0., h_rec-1, h_rec-1]])

ax2 = f1.add_subplot(122)
ax2.scatter(p_B[0,0],p_B[1,0],ec='r', s=markersize, fc='none')
ax2.scatter(p_B[0,1],p_B[1,1],ec='g', s=markersize, fc='none')
ax2.scatter(p_B[0,2],p_B[1,2],ec='b', s=markersize, fc='none')
ax2.scatter(p_B[0,3],p_B[1,3],ec='k', s=markersize, fc='none')

ax2.set(
       xlim = [0, w_rec-1],
       ylim = [0, h_rec-1],
       title='Document rectifié')
ax2.invert_yaxis()

plt.pause(0.1)

def DLT(p_A, p_B):
    p_A_vec = p_A.ravel(order='F')
    M = np.zeros((8,8))
    
    for i in range(4):
        pBx = p_B[0,i]
        pBy = p_B[1,i]
        pAx = p_A[0,i]
        pAy = p_A[1,i]
        
        M_line_1 = [pBx, 0, -pBx*pAx, pBy, 0, -pBy*pAx, 1, 0]
        M_line_2 = [0, pBx, -pBx*pAy, 0, pBy, -pBy*pAy, 0, 1]
        
        M[i*2:(i+1)*2,:] = np.array([M_line_1, M_line_2])
    
    
    
    h = np.linalg.solve(M.T.dot(M), M.T.dot(p_A_vec))
    
    H_AB = np.array([[h[0], h[3], h[6]], [h[1], h[4], h[7]], [h[2], h[5], 1]])
    return H_AB

H_AB = DLT(p_A, p_B)

temp = H_AB.dot(np.vstack([p_B, np.ones((1,4))]))
p_A_pred_x = temp[0,:]/temp[2,:]
p_A_pred_y = temp[1,:]/temp[2,:]

ax1.scatter(p_A_pred_x[0],p_A_pred_y[0],fc='r', s=markersize, marker='x')
ax1.scatter(p_A_pred_x[1],p_A_pred_y[1],fc='g', s=markersize, marker='x')
ax1.scatter(p_A_pred_x[2],p_A_pred_y[2],fc='b', s=markersize, marker='x')
ax1.scatter(p_A_pred_x[3],p_A_pred_y[3],fc='k', s=markersize, marker='x')

#%% Apply transformation

XI,YI = getInterpolationGrids(H_AB, h_rec, w_rec)


I_undist,_ = interp2_bilinear(im, XI, YI)
ax2.imshow(I_undist)
plt.pause(0.1)

Image.fromarray((I_undist*255).astype(np.uint8)).save("im_rect.pdf")

#%% Comparison opencv

H_AB_cv2, mask = cv2.findHomography(p_B.astype(np.float32).T, p_A.astype(np.float32).T)
print(np.abs(H_AB-H_AB_cv2).sum())
